<a href="https://colab.research.google.com/github/BullDF/A4_JSC270/blob/main/notebook.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# JSC270 Assignment 4 by Johnny and Ethan

In [1]:
## Import necessary libraries
from typing import List
import pandas as pd
import numpy as np
import spacy
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Part I: Sentiment Analysis

**a) Consider the training data. What is the balance between the three classes? In other words, what proportion of the observations (in the training set) belong to each class?**

In [2]:
train_url = 'https://raw.githubusercontent.com/3th4novo/JSC270_a4_dataset/main/Corona_NLP_train.csv'
test_url = 'https://raw.githubusercontent.com/3th4novo/JSC270_a4_dataset/main/Corona_NLP_test.csv'

df_train = pd.read_csv(train_url, encoding='latin-1')
df_test = pd.read_csv(test_url, encoding='latin-1')

df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
df_train['Sentiment'].replace({'Extremely Negative': 'Negative', 'Extremely Positive': 'Positive'}, inplace=True)
df_train['Sentiment'].value_counts()

Positive    18046
Negative    15398
Neutral      7713
Name: Sentiment, dtype: int64

In [ ]:
len(df_train['Sentiment'])

41157

**b) Tokenize the tweets. In other words, for each observation, convert the tweet from a single string of running text into a list of individual tokens (possibly with punctuation), splitting on whitespace. The result should be that each observation (tweet) is a list of individual tokens.**

In [5]:
def tokenize(tweet):
   return tweet.split()

df_train['tokens'] = df_train['OriginalTweet'].apply(tokenize)

print(df_train['tokens'].head(5))

0    [@MeNyrbie, @Phil_Gahan, @Chrisitv, https://t....
1    [advice, Talk, to, your, neighbours, family, t...
2    [Coronavirus, Australia:, Woolworths, to, give...
3    [My, food, stock, is, not, the, only, one, whi...
4    [Me,, ready, to, go, at, supermarket, during, ...
Name: tokens, dtype: object


**c) Using a regular expression, remove any URL tokens from each of the observations.**

In [7]:
def remove_url(lst):
  result = [item for item in lst if not re.search('http\S+|www\.\S+', item)]
  return result

df_train['tokens_no_url'] = df_train['tokens'].apply(remove_url)
print(df_train['tokens_no_url'].head(5))

0        [@MeNyrbie, @Phil_Gahan, @Chrisitv, and, and]
1    [advice, Talk, to, your, neighbours, family, t...
2    [Coronavirus, Australia:, Woolworths, to, give...
3    [My, food, stock, is, not, the, only, one, whi...
4    [Me,, ready, to, go, at, supermarket, during, ...
Name: tokens_no_url, dtype: object


**d) Remove all punctuation (,.?!;:’") and special characters(@, #, +, &, =, $, etc). Also, convert all tokens to lowercase only. Can you think of a scenario when you might want to keep some forms of punctuation?**

In [11]:
def remove_punc_spec_lower(lst):
    result = [re.sub('[^\w\s]', '', item) for item in lst]
    lowered = [item.lower() for item in result]
    return lowered

df_train['tokens_no_punc'] = df_train['tokens_no_url'].apply(remove_punc_spec_lower)
print(df_train['tokens_no_punc'].head(5))

0           [menyrbie, phil_gahan, chrisitv, and, and]
1    [advice, talk, to, your, neighbours, family, t...
2    [coronavirus, australia, woolworths, to, give,...
3    [my, food, stock, is, not, the, only, one, whi...
4    [me, ready, to, go, at, supermarket, during, t...
Name: tokens_no_punc, dtype: object


**e) Now stem your tokens. This will have the effect of converting similar word forms into identical tokens (e.g. run, runs, running → run). Please specify which stemmer you use.**

In [13]:
stemmer = PorterStemmer()

def stemming(lst):
  token_stems = [stemmer.stem(w) for w in lst]
  return token_stems

df_train['tokens_stemmed'] = df_train['tokens_no_punc'].apply(stemming)
print(df_train['tokens_stemmed'].head(5))

0            [menyrbi, phil_gahan, chrisitv, and, and]
1    [advic, talk, to, your, neighbour, famili, to,...
2    [coronaviru, australia, woolworth, to, give, e...
3    [my, food, stock, is, not, the, onli, one, whi...
4    [me, readi, to, go, at, supermarket, dure, the...
Name: tokens_stemmed, dtype: object


**f) Lastly, remove stopwords. Using the english stopwords list from nltk, remove these common words from your observations. This list is very long (I think almost 200 words), so remove only the first 100 stopwords in the list.**

In [14]:
sw = stopwords.words('english')[0:99]

def remove_sw(lst):
  tokens_no_sw = []
  tokens_no_sw.append([w for w in lst if w not in sw])
  return tokens_no_sw

df_train['stems_no_sw'] = df_train['tokens_stemmed'].apply(remove_sw)
print(df_train['stems_no_sw'].head(5))

0                    [[menyrbi, phil_gahan, chrisitv]]
1    [[advic, talk, neighbour, famili, exchang, pho...
2    [[coronaviru, australia, woolworth, give, elde...
3    [[food, stock, not, onli, one, empti, pleas, d...
4    [[readi, go, supermarket, dure, covid19, outbr...
Name: stems_no_sw, dtype: object


**g) Now convert your lists of words into vectors of word counts. You may find Scikit-learn’s CountVectorizer useful here. What is the length of your vocabulary?**

**h) Fit a Naive Bayes model to your data. Report the training and test error of the model. Use accuracy as the error metric. Also, report the 5 most probable words in each class, along with their counts. You might find Scikit-learn’s MultinomialNB() transformer useful. Use Laplace smoothing to prevent probabilities of zero.**

**i) Would it be appropriate to fit an ROC curve in this scenario? If yes, explain why. If no, explain why not.**

**j) Redo parts G-H using TF-IDF vectors instead of count vectors. You might find Scikitlearn’s TfidfVectorizer() transformer useful. Report the training and test accuracy. How does this compare to the accuracy using count vectors?**

**k) Recall lemmatization converts each word to its base form, which is a bit stronger than simply taking the stem. Redo parts E-H using TF-IDF vectors instead of count vectors. This time use lemmatization instead of stemming. Report train and test accuracy. How does the accuracy with lemmatization compare to the accuracy with stemming?**

**Bonus: Is the Naive Bayes model generative or discriminative? Explain your response.**